## Regresion Logistica 
### Marco Antonio Jurado 20308


In [390]:
import numpy as np

In [391]:
import csv
with open('framingham.csv', 'r') as file:
    csv_reader = csv.reader(file)
    datacsv = []
    for row in csv_reader:
        datacsv.append(row)

header = datacsv[0]
i = 0
for name in header:
    print(i, name)
    i +=1

0 male
1 age
2 education
3 currentSmoker
4 cigsPerDay
5 BPMeds
6 prevalentStroke
7 prevalentHyp
8 diabetes
9 totChol
10 sysBP
11 diaBP
12 BMI
13 heartRate
14 glucose
15 TenYearCHD


In [392]:
rawDATA = np.genfromtxt('framingham.csv', delimiter=',', skip_header=True)
data = np.array(rawDATA)
print("head \n", data[:5])
print("\n\ntail \n", data[-5:])

head 
 [[  1.    39.     4.     0.     0.     0.     0.     0.     0.   195.
  106.    70.    26.97  80.    77.     0.  ]
 [  0.    46.     2.     0.     0.     0.     0.     0.     0.   250.
  121.    81.    28.73  95.    76.     0.  ]
 [  1.    48.     1.     1.    20.     0.     0.     0.     0.   245.
  127.5   80.    25.34  75.    70.     0.  ]
 [  0.    61.     3.     1.    30.     0.     0.     1.     0.   225.
  150.    95.    28.58  65.   103.     1.  ]
 [  0.    46.     3.     1.    23.     0.     0.     0.     0.   285.
  130.    84.    23.1   85.    85.     0.  ]]


tail 
 [[  1.    50.     1.     1.     1.     0.     0.     1.     0.   313.
  179.    92.    25.97  66.    86.     1.  ]
 [  1.    51.     3.     1.    43.     0.     0.     0.     0.   207.
  126.5   80.    19.71  65.    68.     0.  ]
 [  0.    48.     2.     1.    20.      nan   0.     0.     0.   248.
  131.    72.    22.    84.    86.     0.  ]
 [  0.    44.     1.     1.    15.     0.     0.     0.     0. 

Ahora se separa la X y la Y

In [393]:
X = data[:, 12].reshape(-1, 1)
Y = data[:, -1].reshape(-1, 1)
print(X.shape)
print(Y.shape)

(4238, 1)
(4238, 1)


In [394]:
if np.any(np.isnan(X)):
    print("Faltan datos en X.")
else:
    print("Todo bien en X.")

# Check if there are any NaN values in Y
if np.any(np.isnan(Y)):
    print("Faltan datos en Y.")
else:
    print("Todo bien en Y.")

Faltan datos en X.
Todo bien en Y.


debido a que faltan datos en X se procede a saber que porcentaje de estos es NaN

In [395]:
print('falta un porcentaje del:')
print((np.count_nonzero(np.isnan(X)) * 100) /(X.shape[0]))

falta un porcentaje del:
0.4483246814535158


Se usara SimpleImputer para llenar los datos faltantes con el metodo de mean

In [396]:
from sklearn.impute import SimpleImputer
imp = SimpleImputer(strategy="mean")
X = imp.fit_transform(X)
if np.any(np.isnan(X)):
    print("Faltan datos en X.")
else:
    print("Todo bien en X.")

Todo bien en X.


Ahora ya no tenemos datos faltantes en ambas variables

In [397]:
print(X.shape)
print(Y.shape)

(4238, 1)
(4238, 1)


Procedemos a separar en test y entreno

In [398]:
import random
combinadoZIP = list(zip(X, Y))
random.shuffle(combinadoZIP)

shuffleX, shuffleY = zip(*combinadoZIP)
shuffleX, shuffleY = np.array(shuffleX), np.array(shuffleY)

In [399]:
np.random.seed(54)
shuffle_indices = np.random.permutation(X.shape[0])
X_shuffled = X[shuffle_indices]
Y_shuffled = Y[shuffle_indices]

entreno_temp = int(X.shape[0] * 0.7)

X_train, X_test = X_shuffled[:entreno_temp], X_shuffled[entreno_temp:]
Y_train, Y_test = Y_shuffled[:entreno_temp], Y_shuffled[entreno_temp:]

ahora las funciones que nos sirven para calcular la data donde la funcion de costo es cambiada de la de regresion lineal por la de regresion logistica. Cabe mencionar que es similar pero el detalle del sigmoide es lo que la diferencia.

In [400]:
sigmoid = lambda X, t: 1 / (1 + np.exp(-(X @ t)))
cost = lambda X, y, t: ((sigmoid(X, t) - y) ** 2).sum() / len(y)
grad = lambda X, y, t: 2 * X.T @ (X @ t - y) / len(y)

In [401]:
def regLineal(X, y, t, cost, grad, a=0.1, n=1000, on_step=None): 
    costs = []
    for i in range(n):
        t -= a * grad(X, y, t)
        costs.append(cost(X, y, t))
        if on_step:
            on_step(t)
    
    return t, costs

Procedemos a buscar el polinomio

In [402]:
print(Y_train.shape)
print(Y_test.shape)
print(X_train.shape)
print(X_test.shape)

(2966, 1)
(1272, 1)
(2966, 1)
(1272, 1)


Investigue como funciona Cross Validation de Scikit Learn pues en ocasiones pasadas no llegue a obtener resultados faciles de comprender. Por lo tanto he llegado a comprender el funcionamiento del mismo y obtengo lo siguiente.<br>
fuente utilizada: https://scikit-learn.org/stable/modules/cross_validation.html


In [403]:
def validacion(X, y, k, reg, cost, grad, a=0.1, n=1000):
    scores = []
    temp_size = len(X) // k
    for i in range(k):
        X_test = X[i * temp_size:(i + 1) * temp_size]
        y_test = y[i * temp_size:(i + 1) * temp_size]
        X_train = np.concatenate([X[:i * temp_size], X[(i + 1) * temp_size:]])
        y_train = np.concatenate([y[:i * temp_size], y[(i + 1) * temp_size:]])

        Xr = np.column_stack((np.ones_like(X_train), X_train))
        theta_0 = np.random.rand(Xr.shape[1],1)
        theta_0 = theta_0[~np.isnan(theta_0)].reshape((-1, 1))
        
        t, _ = regLineal(Xr, y_train, theta_0, cost, grad, a=a, n=n)
        print(X_test.shape)
        print(y_test.shape)
        print(t.shape)
        score = cost(X_test, y_test, t)
        scores.append(score)

    promediado = np.mean(scores)
    return promediado

In [404]:
X_shuffled.shape

(4238, 1)

In [405]:
Y_shuffled.shape

(4238, 1)

In [406]:
np.zeros(X_train.shape[1])

array([0.])

In [407]:
k = 5
score = validacion(X_shuffled, Y_shuffled, k, regLineal, cost, grad, a=0.1, n=1000)
print(f"El puntaje promedio de validación cruzada con k={k} es {score}")

(847, 1)
(847, 1)
(2, 1)


C:\Users\marco\AppData\Local\Temp\ipykernel_75772\2081190392.py:1: RuntimeWarning: overflow encountered in exp
  sigmoid = lambda X, t: 1 / (1 + np.exp(-(X @ t)))
C:\Users\marco\AppData\Local\Temp\ipykernel_75772\2081190392.py:3: RuntimeWarning: overflow encountered in matmul
  grad = lambda X, y, t: 2 * X.T @ (X @ t - y) / len(y)
C:\Users\marco\AppData\Local\Temp\ipykernel_75772\2859185945.py:4: RuntimeWarning: invalid value encountered in subtract
  t -= a * grad(X, y, t)


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 2 is different from 1)